# プライベート認証局で証明書を作成する

概要: `server` 上にdockerコンテナでCentOS 7を立ち上げ、その中にCAを構築しサーバー証明書・クライアント証明書を発行する。

詳細なドキュメントは [プライベート認証局で証明書を作成する](https://nii-gakunin-cloud.github.io/sinetstream/docs/userguide/certificate.html) を参照のこと。

## プライベート認証局を構築する

In [1]:
server=server1.example.jp
user=piyo

country="JP"

subject="/C=${country}/ST=Example_State/O=Example_Organization/CN=private-ca"
server_fqdn="broker.example.org"

client_common_name="client0"

### ここで示す手順は CentOS 7 で実行することを前提としている。

In [2]:
ssh ${user}@${server} 'docker run --detach --tty --name sinetstream_ca centos:7'
ssh ${user}@${server} 'docker ps'

c654874e8c7b777f9ffb4ee2ccab2521798665ddc07aced14b602219afe55ce4
CONTAINER ID        IMAGE                            COMMAND                  CREATED             STATUS                  PORTS                                            NAMES
c654874e8c7b        centos:7                         "/bin/bash"              1 second ago        Up Less than a second                                                    sinetstream_ca
33b8b736b890        eclipse-mosquitto:1.6            "/docker-entrypoint.…"   13 days ago         Up 13 days              0.0.0.0:1883->1883/tcp, 0.0.0.0:9001->9001/tcp   mosquitto_mosquitto_1
6adecd62865a        hlebalbau/kafka-manager:stable   "/kafka-manager/bin/…"   3 months ago        Up 3 months                                                              manager_kafka-manager_1
ece7b78ddfb7        zookeeper                        "/docker-entrypoint.…"   10 months ago       Up 3 months             2181/tcp, 2888/tcp, 3888/tcp                     some-zookeepe

### openssl パッケージをインストールする

In [3]:
ssh ${user}@${server} 'docker exec sinetstream_ca yum -y update'

Loaded plugins: fastestmirror, ovl
Determining fastest mirrors
 * base: ftp.riken.jp
 * extras: ftp.riken.jp
 * updates: ftp.riken.jp
Resolving Dependencies
--> Running transaction check
---> Package binutils.x86_64 0:2.27-41.base.el7 will be updated
---> Package binutils.x86_64 0:2.27-41.base.el7_7.3 will be an update
---> Package ca-certificates.noarch 0:2018.2.22-70.0.el7_5 will be updated
---> Package ca-certificates.noarch 0:2019.2.32-76.el7_7 will be an update
---> Package curl.x86_64 0:7.29.0-54.el7 will be updated
---> Package curl.x86_64 0:7.29.0-54.el7_7.2 will be an update
---> Package device-mapper.x86_64 7:1.02.158-2.el7 will be updated
---> Package device-mapper.x86_64 7:1.02.158-2.el7_7.2 will be an update
---> Package device-mapper-libs.x86_64 7:1.02.158-2.el7 will be updated
---> Package device-mapper-libs.x86_64 7:1.02.158-2.el7_7.2 will be an update
---> Package hostname.x86_64 0:3.13-3.el7 will be updated
---> Package hostname.x86_64 0:3.13-3.el7_7.1 will be an upda

In [4]:
ssh ${user}@${server} 'docker exec sinetstream_ca yum -y install openssl'

Loaded plugins: fastestmirror, ovl
Loading mirror speeds from cached hostfile
 * base: ftp.riken.jp
 * extras: ftp.riken.jp
 * updates: ftp.riken.jp
Resolving Dependencies
--> Running transaction check
---> Package openssl.x86_64 1:1.0.2k-19.el7 will be installed
--> Processing Dependency: make for package: 1:openssl-1.0.2k-19.el7.x86_64
--> Running transaction check
---> Package make.x86_64 1:3.82-24.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package          Arch            Version                   Repository     Size
Installing:
 openssl          x86_64          1:1.0.2k-19.el7           base          493 k
Installing for dependencies:
 make             x86_64          1:3.82-24.el7             base          421 k

Transaction Summary
Install  1 Package (+1 Dependent package)

Total download size: 914 k
Installed size: 1.9 M
--------------------------------------------------------------------------------
Total                                  

### 証明書や秘密鍵などを格納するディレクトリを作成する

In [5]:
ssh ${user}@${server} 'docker exec sinetstream_ca mkdir -p /etc/pki/CA/certs /etc/pki/CA/crl /etc/pki/CA/newcerts /etc/pki/CA/private'

### プライベート認証局のために必要となる設定を行う

* unique_subject
    * CA証明書のロールオーバーを簡単にするために no を指定する
* copy_extensions
    * 証明書のリクエストが SAN(subjectAltName)をコピーできるようにするためにcopyを指定する

In [6]:
ssh ${user}@${server} 'docker exec sinetstream_ca sed --in-place "/unique_subject/s/^.*/unique_subject = no/;/copy_extensions/s/^.*/copy_extensions = copy/" /etc/pki/tls/openssl.cnf'

プライベート認証局が署名した証明書を記録するためのファイル index.txt を作成する

In [7]:
ssh ${user}@${server} 'docker exec sinetstream_ca touch /etc/pki/CA/index.txt'

### CA証明書のCSRと秘密鍵を作成する

In [8]:

ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl req -new \
                -keyout /etc/pki/CA/private/cakey.pem \
                -out /etc/pki/CA/careq.pem \
                -nodes \
                -subj ${subject}"
ssh ${user}@${server} 'docker exec sinetstream_ca ls -l /etc/pki/CA/private/cakey.pem'

Generating a 2048 bit RSA private key
................................................................................+++
.....................................................+++
writing new private key to '/etc/pki/CA/private/cakey.pem'
-----
-rw-r--r-- 1 root root 1704 Mar 24 09:56 /etc/pki/CA/private/cakey.pem


### 自己署名によるCA証明書を作成する

In [9]:
ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl ca -batch \
               -in /etc/pki/CA/careq.pem \
               -selfsign \
               -extensions v3_ca \
               -keyfile /etc/pki/CA/private/cakey.pem \
               -days 3650 \
               -create_serial \
               -out /etc/pki/CA/cacert.pem"
ssh ${user}@${server} "docker exec sinetstream_ca ls -l /etc/pki/CA/cacert.pem"

Using configuration from /etc/pki/tls/openssl.cnf
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number:
            c5:b2:cf:5c:ab:23:77:90
        Validity
            Not Before: Mar 24 09:56:33 2020 GMT
            Not After : Mar 22 09:56:33 2030 GMT
        Subject:
            countryName               = JP
            stateOrProvinceName       = Example_State
            organizationName          = Example_Organization
            commonName                = private-ca
        X509v3 extensions:
            X509v3 Subject Key Identifier: 
                B6:0A:CE:3D:55:44:0A:2C:9E:B1:29:4E:7F:C5:DB:4C:87:4B:06:DB
            X509v3 Authority Key Identifier: 
                keyid:B6:0A:CE:3D:55:44:0A:2C:9E:B1:29:4E:7F:C5:DB:4C:87:4B:06:DB

            X509v3 Basic Constraints: 
                CA:TRUE
Certificate is to be certified until Mar 22 09:56:33 2030 GMT (3650 days)

Write out database with 1 new entries
Data Base Updated
-

作成したCA証明書/etc/pki/CA/cacert.pemの内容を確認する

In [10]:
ssh ${user}@${server} "docker exec sinetstream_ca openssl x509 -in /etc/pki/CA/cacert.pem -noout -text"

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            c5:b2:cf:5c:ab:23:77:90
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=JP, ST=Example_State, O=Example_Organization, CN=private-ca
        Validity
            Not Before: Mar 24 09:56:33 2020 GMT
            Not After : Mar 22 09:56:33 2030 GMT
        Subject: C=JP, ST=Example_State, O=Example_Organization, CN=private-ca
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (2048 bit)
                Modulus:
                    00:ee:1b:a6:de:5b:2d:df:d8:2d:d7:43:94:b7:14:
                    13:57:38:50:ca:9f:1c:fc:96:6c:9d:9b:03:2c:50:
                    5e:df:11:23:28:d1:6d:6d:1b:3f:ef:36:1c:2e:b1:
                    bb:5a:8d:81:1e:e2:6c:24:bb:35:95:bf:27:48:c5:
                    29:91:06:f0:a5:ec:00:3f:35:d2:c5:2b:31:ef:83:
                    32:ba:99:dd:5f:c2:6a:4a:29:b2:78:ec:ae:60:98:
                    eb:3d:79:69

### 作成したCA証明書を手元にコピーする。

In [19]:
ssh ${user}@${server} 'docker exec sinetstream_ca cat /etc/pki/CA/cacert.pem' > cacert.pem
ls -l cacert.pem

-rw-r--r-- 1 jovyan users 4349 Mar 27 14:38 cacert.pem


## サーバ証明書の秘密鍵、証明書を作成する

### 証明書のSAN(subjectAltName)にサーバのホスト名を追加するための設定

In [11]:
ssh ${user}@${server} "docker exec sinetstream_ca /bin/sh -c 'cat >>/etc/pki/tls/openssl.cnf'" << EOF
[ req ]
req_extensions = v3_req

[ v3_req ]
subjectAltName = @alt_names

[ alt_names ]
DNS = ${server_fqdn}
EOF

### サーバの秘密鍵とサーバ証明書のCSR(Certificate Signing Request)を作成する

In [12]:
ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl req -new \
                -keyout /etc/pki/CA/private/broker.key \
                -out /etc/pki/CA/broker.csr \
                -nodes \
                -subj /C=${country}/CN=${server_fqdn}"

Generating a 2048 bit RSA private key
...+++
...........+++
writing new private key to '/etc/pki/CA/private/broker.key'
-----


### CA証明書で署名をおこない、サーバ証明書を作成する

In [13]:
ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl ca -batch \
               -keyfile /etc/pki/CA/private/cakey.pem \
               -cert /etc/pki/CA/cacert.pem \
               -in /etc/pki/CA/broker.csr \
               -out /etc/pki/CA/certs/broker.crt \
               -policy policy_anything"

Using configuration from /etc/pki/tls/openssl.cnf
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number:
            c5:b2:cf:5c:ab:23:77:91
        Validity
            Not Before: Mar 24 09:56:59 2020 GMT
            Not After : Mar 24 09:56:59 2021 GMT
        Subject:
            countryName               = JP
            commonName                = broker.example.org
        X509v3 extensions:
            X509v3 Basic Constraints: 
                CA:FALSE
            Netscape Comment: 
                OpenSSL Generated Certificate
            X509v3 Subject Key Identifier: 
                5C:68:29:27:7A:84:4E:B6:32:99:01:6A:8C:D3:B9:EE:D9:D4:AC:E3
            X509v3 Authority Key Identifier: 
                keyid:B6:0A:CE:3D:55:44:0A:2C:9E:B1:29:4E:7F:C5:DB:4C:87:4B:06:DB

Certificate is to be certified until Mar 24 09:56:59 2021 GMT (365 days)

Write out database with 1 new entries
Data Base Updated


作成したサーバ証明書/etc/pki/CA/certs/broker.crtの内容を確認する

In [14]:
ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl x509 -in /etc/pki/CA/certs/broker.crt -noout -text"

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            c5:b2:cf:5c:ab:23:77:91
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=JP, ST=Example_State, O=Example_Organization, CN=private-ca
        Validity
            Not Before: Mar 24 09:56:59 2020 GMT
            Not After : Mar 24 09:56:59 2021 GMT
        Subject: C=JP, CN=broker.example.org
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (2048 bit)
                Modulus:
                    00:ef:29:c2:32:89:21:7b:8f:5a:b2:82:fe:df:df:
                    eb:a1:89:a9:b7:d3:7d:3d:5d:39:9e:c7:72:11:5f:
                    43:63:f0:bd:d6:07:1d:f6:00:52:fd:e2:88:3a:e7:
                    85:2f:b7:f8:51:db:2f:c8:2e:19:00:9d:3e:c9:fc:
                    95:d6:8d:b6:8c:35:0f:50:4c:6c:6f:fa:23:d7:4c:
                    97:7a:ec:87:98:38:1e:96:aa:05:2f:ad:76:16:77:
                    0d:a0:2a:5d:ae:b3:18:ea:3d:93:83:63:6c:61:f1:

### 作成したサーバ秘密鍵とサーバ証明書を手元にコピーする

サーバ秘密鍵にパスフレーズを設定してないので盗難に注意する。

In [20]:
ssh ${user}@${server} 'docker exec sinetstream_ca cat /etc/pki/CA/private/broker.key ' > broker.key
ssh ${user}@${server} 'docker exec sinetstream_ca cat /etc/pki/CA/certs/broker.crt' > broker.crt
ls -l broker.key broker.crt

-rw-r--r-- 1 jovyan users 4389 Mar 27 15:01 broker.crt
-rw-r--r-- 1 jovyan users 1704 Mar 27 15:01 broker.key


## クライアント認証のための秘密鍵、証明書を作成する

### クライアント証明書のCSRと秘密鍵を作成する

In [15]:


ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl req -new \
                -keyout /etc/pki/CA/private/${client_common_name}.key \
                -out /etc/pki/CA/${client_common_name}.csr \
                -nodes \
                -subj /C=${country}/CN=${client_common_name}"

Generating a 2048 bit RSA private key
..............................................................................................+++
..............+++
writing new private key to '/etc/pki/CA/private/client0.key'
-----


### CA証明書で署名をおこない、クライアント証明書を作成する

In [16]:
ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl ca -batch \
               -keyfile /etc/pki/CA/private/cakey.pem \
               -cert /etc/pki/CA/cacert.pem \
               -in /etc/pki/CA/client0.csr \
               -out /etc/pki/CA/certs/${client_common_name}.crt \
               -policy policy_anything"

Using configuration from /etc/pki/tls/openssl.cnf
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number:
            c5:b2:cf:5c:ab:23:77:92
        Validity
            Not Before: Mar 24 09:57:22 2020 GMT
            Not After : Mar 24 09:57:22 2021 GMT
        Subject:
            countryName               = JP
            commonName                = client0
        X509v3 extensions:
            X509v3 Basic Constraints: 
                CA:FALSE
            Netscape Comment: 
                OpenSSL Generated Certificate
            X509v3 Subject Key Identifier: 
                35:F6:D3:43:27:74:2B:D4:45:DD:61:46:1D:29:9F:F4:3F:68:E0:DD
            X509v3 Authority Key Identifier: 
                keyid:B6:0A:CE:3D:55:44:0A:2C:9E:B1:29:4E:7F:C5:DB:4C:87:4B:06:DB

Certificate is to be certified until Mar 24 09:57:22 2021 GMT (365 days)

Write out database with 1 new entries
Data Base Updated


作成したクライアント証明書/etc/pki/CA/certs/client0.crtの内容を確認する

In [17]:
ssh ${user}@${server} "docker exec sinetstream_ca \
    openssl x509 -in /etc/pki/CA/certs/${client_common_name}.crt -noout -text"

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            c5:b2:cf:5c:ab:23:77:92
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=JP, ST=Example_State, O=Example_Organization, CN=private-ca
        Validity
            Not Before: Mar 24 09:57:22 2020 GMT
            Not After : Mar 24 09:57:22 2021 GMT
        Subject: C=JP, CN=client0
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (2048 bit)
                Modulus:
                    00:b9:19:24:59:70:05:6a:a9:be:40:f6:bd:f6:0a:
                    74:77:8f:e2:d9:66:56:30:30:79:01:ce:82:e4:bf:
                    cd:cc:c7:04:74:4a:f4:83:e5:c9:04:96:d9:98:bd:
                    ce:39:71:ff:83:34:39:7e:26:dd:24:7d:af:b2:25:
                    1d:7f:2b:32:45:89:28:a9:e0:f1:13:79:60:70:ae:
                    5f:e9:c5:29:ea:08:f9:df:0b:20:2f:cd:7f:9b:0f:
                    9f:14:5a:91:5d:e2:7c:31:75:b7:90:cb:ce:45:ec:
          